In [2]:
!pip install tensorflow==2.18.0

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine, Table, MetaData
from tensorflow.keras.models import load_model

# Load models and files
model = load_model('models/disease_prediction_model.h5')  # Path to .h5 file
with open('models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('models/binary_features.pkl', 'rb') as f:
    binary_features = pickle.load(f)

# Setup database connection
db_params = {
    'dbname': 'disease_symptom_db',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'
}
connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
engine = create_engine(connection_string)
metadata = MetaData()
metadata.reflect(bind=engine)  # Load table metadata

# Map tables for direct queries
Symptoms = Table('symptoms_tb', metadata, autoload_with=engine)
Diseases = Table('diseases_tb', metadata, autoload_with=engine)
DiseaseSymptom = Table('disease_symptom_tb', metadata, autoload_with=engine)

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
# Simulated user input
selectedSymptom_ids = ['C0085593', 'C0746619', 'C0030193', 'C0004093', 'C0004604']
print("Selected Symptom IDs:", selectedSymptom_ids)


In [ ]:
from sqlalchemy.orm import Session

session = Session(engine)
symptom_names = []
for symptom_id in selectedSymptom_ids:
    result = session.query(Symptoms).filter_by(symptom_id=symptom_id).first()
    if result:
        symptom_names.append(result.s_name)
    else:
        print(f"Symptom ID {symptom_id} not found in the database.")
session.close()
print("Symptom Names:", symptom_names)


In [ ]:
# Create test input
test_input = pd.DataFrame([{symptom_id: 1 for symptom_id in selectedSymptom_ids}], columns=binary_features).fillna(0)

# Validate input alignment
print("Test Input DataFrame:\n", test_input.head())
print("Binary Features:", binary_features)

# Scale test input
test_input_scaled = scaler.transform(test_input)
print("Scaled Test Input:\n", test_input_scaled)


In [ ]:
# Predict disease probabilities
probabilities = model.predict(test_input_scaled)
print("Predicted Probabilities:", probabilities)


In [ ]:
top_indices = np.argsort(probabilities[0])[-3:][::-1]
print("Top Indices:", top_indices)

# Query disease_symptom_hotcoded structure
session = Session(engine)
symptom_disease_data = session.query(DiseaseSymptom).all()
symptom_disease_df = pd.DataFrame([
    {"disease_id": item.disease_id, "symptom_id": item.symptom_id}
    for item in symptom_disease_data
])

# Recreate disease_symptom_hotcoded
binary_features = pd.get_dummies(symptom_disease_df['symptom_id'])
disease_symptom_hotcoded = pd.concat(
    [symptom_disease_df['disease_id'], binary_features],
    axis=1
).groupby('disease_id').sum().reset_index()
print("Disease Symptom Hotcoded:\n", disease_symptom_hotcoded.head())

# Get top matches
matches_response = []
for index in top_indices:
    if index >= len(disease_symptom_hotcoded):
        print(f"Index {index} is out of bounds for disease_symptom_hotcoded.")
        continue

    try:
        row = disease_symptom_hotcoded.iloc[index]
        disease_id = row['disease_id']

        # Lookup disease details
        disease_result = session.query(Diseases).filter_by(disease_id=disease_id).first()
        if disease_result:
            associated_symptoms = session.query(DiseaseSymptom).filter_by(disease_id=disease_id).all()
            symptom_list = [symptom.s_name for symptom in associated_symptoms]

            # Build match entry
            matches_response.append({
                "d_name": disease_result.d_name,
                "description": disease_result.description,
                "s_name": symptom_list,
                "probability": f"{probabilities[0][index] * 100:.2f}%"
            })
    except Exception as e:
        print(f"Error processing index {index}: {str(e)}")
session.close()
print("Matches Response:", matches_response)
